In [169]:
import numpy as np
import pandas as pd
import math
import matplotlib.animation as animation
import matplotlib as plt
import seaborn as sns

In [133]:
# Parametros para função de mobilidade
MD = 20 # Distância máxima do salto - variações 0=nenhuma 5,10 e 20=Todo o grid 
IM = 20 # Proporção de individuos móveis - variações: 0,20%,50% e 100% 
neighbors_type = 'Moore'
radius = 1

rows, cols = 20, 20
n_cells = rows * cols  
rows_grid, cols_grid = np.mgrid[0:rows, 0:cols]  # Gera um grid
row_indices = rows_grid.flatten()  # Transforma em array 1D [0, 0, ..., 19, 19]
col_indices = cols_grid.flatten()  

pop = np.concatenate([np.zeros(n_cells*95//100), np.ones(n_cells*5//100) * 2]).astype(int)  # 95% da populacao é S e 5% é I 
# Estados possíveis 0=S, 1=E, 2=I, 3=R, 4=Q 
# Cria o dataframe com a população e seus Parametros individuais
data = {
    'row': row_indices,
    'col' : col_indices,
    'status': np.random.permutation(pop).astype(int),
    'learningCapability': np.random.uniform(0.5, 1.5, n_cells),  # aleatorio entre 0.5 e 1.5
    'transferCapability': np.random.uniform(0.5, 1.5, n_cells),  # aleatorio entre 0.5 e 1.5
    'baseForgettingRate': np.random.uniform(0.1, 0.3, n_cells),  # aleatorio entre 0.1 e 0.3
    'baseQuittingRate': np.random.uniform(0.1, 0.3, n_cells),    # aleatorio entre 0.1 e 0.3
    'timesContactorD': np.zeros(n_cells).astype(int)
}
name_cols = ['row', 'col','status', 'learningCapability', 'transferCapability','baseForgettingRate', 'baseQuittingRate', 'timesContactorD']

# Create DataFrame
cellularGrid = pd.DataFrame(data)
# columns to have row and col first
cellularGrid = cellularGrid[name_cols]
# Display the first few rows
print(cellularGrid.head())

   row  col  status  learningCapability  transferCapability  \
0    0    0       0            0.999011            0.610018   
1    0    1       0            0.812823            0.784882   
2    0    2       0            1.437176            1.356245   
3    0    3       0            0.989045            1.240187   
4    0    4       0            1.225868            1.295045   

   baseForgettingRate  baseQuittingRate  timesContactorD  
0            0.225759          0.163119                0  
1            0.252749          0.237776                0  
2            0.159058          0.207876                0  
3            0.252968          0.146041                0  
4            0.147297          0.242151                0  


In [134]:
# Verify the shape and status distribution
print("\nDataFrame shape:", cellularGrid.shape)
print("\nStatus distribution:")
print(cellularGrid['status'].value_counts())

status_matrix = cellularGrid.pivot(index='row', columns='col', values='status')
print(status_matrix)


DataFrame shape: (400, 8)

Status distribution:
status
0    380
2     20
Name: count, dtype: int64
col  0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  \
row                                                                           
0     0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   
1     0   0   0   2   0   0   0   0   0   0   0   0   0   2   0   0   2   0   
2     0   0   0   2   0   0   0   0   2   0   0   0   0   0   0   0   0   0   
3     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   
4     0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   
5     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
6     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
7     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
8     0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   
9     0   0   0   0   0   0   0

In [146]:
def mobility(df, MD, IM, teste=0):
    # Copy DataFrame to avoid modifying the original
    new_df = df.copy()

    max_row = df['row'].max()
    max_col = df['col'].max()

    if MD >= max_row:    
        max_rowMD = max_row
        max_colMD = max_col
    else:
        max_rowMD = MD
        max_colMD = MD

    columns_to_swap = list(df.columns) 
    columns_to_swap.remove('row')
    columns_to_swap.remove('col')
       
    # Calculate IMGrid: number of cells to swap = % da população afetada no movimento
    IMGrid = (n_cells * IM) // 100
    
    # Perform swaps
    for _ in range(IMGrid):
        if MD == 0:
            break  # No swaps if MD is 0

        # Randomly select first cell (i, j)
        i = np.random.randint(0, max_row)
        j = np.random.randint(0, max_col)
        
        # Randomly select second cell (mdI, mdJ) within MDGrid  ####### REVER verificar se precisa ser a MD do ponto sorteado???
        mdI = np.random.randint(0, max_rowMD)
        mdJ = np.random.randint(0, max_colMD)
        while mdI == i and mdJ == j:  # Ensure different cells
            mdI = np.random.randint(0, max_rowMD)
            mdJ = np.random.randint(0, max_colMD)
        
        # Find indices in DataFrame
        idx1 = new_df[(new_df['row'] == i) & (new_df['col'] == j)].index
        idx2 = new_df[(new_df['row'] == mdI) & (new_df['col'] == mdJ)].index
        
        if idx1.empty or idx2.empty:
            continue  # Skip if indices not found
        
        idx1, idx2 = idx1[0], idx2[0]    
        if idx1 == teste or idx2 == teste:   
            print('Trocou------------------',idx1,idx2)
        # Swap all attributes except row and col
        temp = new_df.loc[idx1, columns_to_swap].copy()
        new_df.loc[idx1, columns_to_swap] = new_df.loc[idx2, columns_to_swap]
        new_df.loc[idx2, columns_to_swap] = temp
    
    return new_df    

In [157]:
teste = 35
print(cellularGrid.iloc[teste])
print("------------------------------------")
df = mobility(cellularGrid,MD,IM,teste)
print(df.iloc[teste])
print("------------------------------------")
print(cellularGrid.iloc[teste])

row                    1.000000
col                   15.000000
status                 0.000000
learningCapability     1.343191
transferCapability     1.360716
baseForgettingRate     0.262394
baseQuittingRate       0.143405
timesContactorD        0.000000
Name: 35, dtype: float64
------------------------------------
Trocou------------------ 348 35
Trocou------------------ 35 20
row                    1.000000
col                   15.000000
status                 0.000000
learningCapability     1.393980
transferCapability     0.741919
baseForgettingRate     0.125496
baseQuittingRate       0.210362
timesContactorD        0.000000
Name: 35, dtype: float64
------------------------------------
row                    1.000000
col                   15.000000
status                 0.000000
learningCapability     1.343191
transferCapability     1.360716
baseForgettingRate     0.262394
baseQuittingRate       0.143405
timesContactorD        0.000000
Name: 35, dtype: float64


In [158]:
def neighbors(df, point, radius=2, neighbors_type='Moore'):
    row, col = point
    neighbor_indices =[]
    
    if neighbors_type not in ['Moore', 'VonNeumann']:
        raise ValueError("Escolha Moore ou VonNeumann")
    
    for dx in range(- radius, radius + 1):
        for dy in range(- radius, radius + 1):
            if dx == 0 and dy == 0:
                continue  # ignora a própria célula

            nx = (row + dx) % rows
            ny = (col + dy) % cols
            
            if neighbors_type == "VonNeumann":
                dist = abs(dx) + abs(dy)  # distância de Manhattan
                if dist <= radius:
                    neighbor_indices.append([df[(df['row'] == nx) & (df['col'] == ny)].index,dist])

            elif neighbors_type == "Moore":
                dist = math.sqrt(dx**2 + dy**2)  # distância Euclidiana
                # Moore inclui todas as células no quadrado
                neighbor_indices.append([df[(df['row'] == nx) & (df['col'] == ny)].index,dist])
        
    return neighbor_indices

In [159]:
# Test neighbors function
point = (10, 10)  # Center of the grid
moore_neighbors = neighbors(cellularGrid, point, radius=1, neighbors_type='Moore')
von_neumann_neighbors = neighbors(cellularGrid, point, radius=1, neighbors_type='VonNeumann')

print(moore_neighbors)
print(von_neumann_neighbors)

[[Index([189], dtype='int64'), 1.4142135623730951], [Index([190], dtype='int64'), 1.0], [Index([191], dtype='int64'), 1.4142135623730951], [Index([209], dtype='int64'), 1.0], [Index([211], dtype='int64'), 1.0], [Index([229], dtype='int64'), 1.4142135623730951], [Index([230], dtype='int64'), 1.0], [Index([231], dtype='int64'), 1.4142135623730951]]
[[Index([190], dtype='int64'), 1], [Index([209], dtype='int64'), 1], [Index([211], dtype='int64'), 1], [Index([230], dtype='int64'), 1]]


In [160]:
# Estados possíveis 0=S, 1=E, 2=I, 3=R, 4=Q 
def transition_S_to_E(df, point):
    row, col = point

    # LearningCapability do ponto atual
    idx_current = df[(df['row'] == row) & (df['col'] == col)].index[0]
    lc_current = df.loc[idx_current, 'learningCapability']
    status_current = df.loc[idx_current, 'status']

    # Se não está no estado S (0), não faz transição
    if status_current != 0:
        return None, 0  # sem transição

    # Obter vizinhos com suas distâncias
    neighbor_data = neighbors(df, point, radius=radius, neighbors_type=neighbors_type)

    max_tax = 0
    candidate_index = None

    for indices, dist in neighbor_data:
        if len(indices) == 0 or dist == 0:
            continue
        idx = indices[0]  # índice do vizinho no DataFrame

        status_vizinho = df.loc[idx, 'status']
        if status_vizinho == 2:  # status = I
            lc_vizinho = df.loc[idx, 'learningCapability']
            tax = (1 / dist) * math.sqrt(lc_current * lc_vizinho)

            if tax > max_tax:
                max_tax = tax
                candidate_index = idx

    # Se encontrou pelo menos um vizinho com status 2, faz transição
    if candidate_index is not None:
        df.at[idx_current, 'status'] = 1  # transita de S (0) para E (1)

    return candidate_index, max_tax

def apply_transition_S_to_E(df):
    pontos_S = df[df['status'] == 0][['row', 'col']].values
    transicoes = 0

    for ponto in pontos_S:
        ponto_tuple = tuple(ponto)
        candidato, taxa = transition_S_to_E(df, ponto_tuple)
        if candidato is not None:
            transicoes += 1

    #print(f'Total de transições S → E realizadas: {transicoes}')


In [ ]:
def transition_F(df):
    indices_E = df[df['status'] == 1].index

    for idx in indices_E:
        # Número de vezes que o indivíduo se tornou contactor (D)
        df.at[idx, 'timesContactorD'] += 1
        D = df.at[idx, 'timesContactorD']

        # Taxas base
        R_base = df.at[idx, 'baseForgettingRate']
        Q_base = df.at[idx, 'baseQuittingRate']

        # Ajuste com reforço de aprendizado
        Rl = R_base ** (D + 1)
        Ql = Q_base ** (D + 1)
        Il = 1 - Rl - Ql

        # Normalização em caso de erro numérico
        if Il < 0:
            Il = 0
            total = Rl + Ql
            Rl /= total
            Ql /= total

        # Sorteio do destino
        r = np.random.rand()
        if r < Ql:
            df.at[idx, 'status'] = 4  # Q - Quitter
            df.at[idx, 'timesContactorD'] = 0
        elif r < Ql + Rl:
            df.at[idx, 'status'] = 3  # R - Forgetter
            # Aumenta LC: LC = LC * sqrt(D)
            df.at[idx, 'learningCapability'] *= np.sqrt(D)
        else:
            df.at[idx, 'status'] = 2  # I - Disseminator
            # TransferCapability não muda

In [ ]:
def simular_difusao(df, max_iter=50, press=True):
    history_stats = [] # resetar histórico

    for passo in range(1, max_iter + 1):
        status_antes = df['status'].value_counts().to_dict()

        apply_transition_S_to_E(df)
        transition_F(df)
        df = mobility(df,MD,IM)

        status_depois = df['status'].value_counts().to_dict()

        # Salva os dados para gerar estatísticas
        history_stats.append({
            'ciclo': passo,
            **{f'status_{s}': status_depois.get(s, 0) for s in range(5)}
        })

        if press:
            print(f'\n🌀 Ciclo {passo}')
            print(f'Estatísticas:')
            for s in range(5):
                antes = status_antes.get(s, 0)
                depois = status_depois.get(s, 0)
                print(f'  Estado {s}: {antes} → {depois}')

        if status_antes == status_depois:
            print(f'⛔️ Simulação encerrada no passo {passo}.')
            break
    return history_stats

In [165]:
df = cellularGrid.copy()
history = simular_difusao(df, max_iter=50, press=False)

Trocou------------------ 0 354
Trocou------------------ 44 0
Trocou------------------ 0 256
Trocou------------------ 303 0
Trocou------------------ 104 0
Trocou------------------ 294 0
Trocou------------------ 0 371
Trocou------------------ 310 0
Trocou------------------ 176 0
Trocou------------------ 0 85
Trocou------------------ 284 0
Trocou------------------ 374 0
⛔️ Simulação encerrada no passo 22.


In [170]:
def plot_estados_por_tempo(history):
    df_hist = pd.DataFrame(history)
    plt.figure(figsize=(10,6))
    
    for s in range(5):
        plt.plot(df_hist['ciclo'], df_hist[f'status_{s}'], label=f'Status {s}')
    
    plt.xlabel('Ciclo')
    plt.ylabel('Número de indivíduos')
    plt.title('Evolução dos estados ao longo do tempo')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_mapa_de_status(df, titulo='Distribuição Espacial dos Estados'):
    grid = np.zeros((rows, cols))
    for _, row in df.iterrows():
        grid[int(row['row']), int(row['col'])] = row['status']
    
    plt.figure(figsize=(6,6))
    cmap = sns.color_palette("Set2", n_colors=5)
    sns.heatmap(grid, square=True, cbar=True, cmap=cmap, linewidths=0.2, linecolor='gray')
    plt.title(titulo)
    plt.xlabel('Colunas')
    plt.ylabel('Linhas')
    plt.show()